In [1]:
# !pip install bert-for-tf2

In [12]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from bert import bert_tokenization
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [13]:
train_df = pd.read_csv("resources/train.csv")
test_df = pd.read_csv("resources/test.csv")

In [66]:
# Choose BERT preprocess model
bert_preprocess_model = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'

# Setup tokenizer
tokenizer = bert_tokenization.FullTokenizer(vocab_file,do_lower_case=True)

In [67]:
# Testing tokenizer
test_text = 'An earthquake of 4.3 magnitude was recorded in Emeryville, CA'
tokenizer.tokenize(test_text)

['an',
 'earthquake',
 'of',
 '4',
 '.',
 '3',
 'magnitude',
 'was',
 'recorded',
 'in',
 'emery',
 '##ville',
 ',',
 'ca']

In [68]:
# Build helper functions for BERT encoding
def encoder(texts, tokenizer, max_len=512):
    all_token_ids = []
    all_masks = []
    all_segment_ids = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        pad_masks = [1] * len(input_sequence) + [0]* pad_len

        token_ids = tokenizer.convert_tokens_to_ids(input_sequence)
        token_ids += [0] * pad_len
        segment_ids = [0] * max_len
        
        all_token_ids.append(token_ids)
        all_masks.append(pad_masks)
        all_segment_ids.append(segment_ids)
        
        return np.array(all_token_ids), np.array(all_masks), np.array(all_segment_ids)

In [69]:
# Test encoder helper function
encoder([test_text],tokenizer,8)

(array([[ 101, 2019, 8372, 1997, 1018, 1012, 1017,  102]]),
 array([[1, 1, 1, 1, 1, 1, 1, 1]]),
 array([[0, 0, 0, 0, 0, 0, 0, 0]]))

In [78]:
def build_bert_model(bert_layer,max_len=512):
    # Set input into the bert layer
    input_word_ids = Input(shape=(max_len,),dtype=tf.int32,name="input_word_ids")
    input_mask = Input(shape=(max_len,),dtype=tf.int32,name="input_mask")
    segment_ids = Input(shape=(max_len,),dtype=tf.int32,name="segment_ids")
    
    # Output from bert layer
    cls_out = bert_layer([input_word_ids,input_mask,segment_ids])[:,0,:]
    out = Dense(1,activation='sigmoid')(cls_out)
    
    # Create model with input and output
    
    return model

In [79]:
# Create layer with chosen BERT model
keras_bert_layer = hub.KerasLayer(bert_preprocess_model, trainable=True, name='keras_bert_layer')

# Build BERT model
bert_model = build_bert_model(keras_bert_layer)
bert_model.summary()

ValueError: in user code:

    /usr/local/anaconda3/lib/python3.8/site-packages/tensorflow_hub/keras_layer.py:237 call  *
        result = smart_cond.smart_cond(training,
    /usr/local/anaconda3/lib/python3.8/site-packages/tensorflow/python/saved_model/load.py:664 _call_attribute  **
        return instance.__call__(*args, **kwargs)
    /usr/local/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py:885 __call__
        result = self._call(*args, **kwds)
    /usr/local/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py:933 _call
        self._initialize(args, kwds, add_initializers_to=initializers)
    /usr/local/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/def_function.py:759 _initialize
        self._stateful_fn._get_concrete_function_internal_garbage_collected(  # pylint: disable=protected-access
    /usr/local/anaconda3/lib/python3.8/site-packages/tensorflow/python/eager/function.py:3066 _get_concrete_function_in

In [ ]:
# Setup vocabulary file 
vocab_file = keras_bert_layer.resolved_object.vocab_file.asset_path.numpy()